In [65]:
import pandas as pd
import numpy as np
import os
import librosa
import pyaudio
import wave
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping

In [66]:
DIR = 'team/' # 'alexa/'
speakers = os.listdir(DIR)
# speakers = speakers[:4]

df = pd.DataFrame(columns=['filepath', 'speaker'])
for speaker in speakers:
    files = os.listdir(DIR+'{}/'.format(speaker))
    for file in files:
        filepath = DIR+ '{}/{}'.format(speaker, file)
        df = df.append({'filepath':filepath, 'speaker':speaker}, ignore_index=True)
print(len(speakers))
df.head() # this is just a print statement

4


,filepath,speaker
0,team/alastair/1.wav,alastair
1,team/alastair/2.wav,alastair
2,team/alastair/3.wav,alastair
3,team/alastair/5.wav,alastair
4,team/alex/1.wav,alex


In [67]:

train, test = train_test_split(df, test_size = 0.29, stratify = df['speaker'])
# the stratify parameter makes the function split data evenly over the speakers column
# this is so we dont get all files of the same speaker in the test set and not the training set
train.head()

,filepath,speaker
8,team/eli/1.wav,eli
12,team/harley/1.wav,harley
2,team/alastair/3.wav,alastair
4,team/alex/1.wav,alex
6,team/alex/3.wav,alex


In [68]:
MFCCS = 12

def extract_features(filename):
    
    X, sample_rate = librosa.load(filename, res_type='kaiser_fast')
    
    # librosa returns an array of 40 arrays, one for each mfcc
    # np.mean takes the mean of each array, so we will be left with an array of size 40
    # the n_mfcc=40 parameter means return 40 mfccs
    mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=MFCCS)
    
    mfccs_mean = np.mean(mfccs.T, axis=0)
    
    # plp = librosa.beat.plp(y=X, sr=22050, onset_envelope=None, hop_length=512, win_length=99, tempo_min=30, tempo_max=300, prior=None)
    
    return mfccs_mean.tolist()#  + plp.tolist()

In [69]:
# apply the extract features to every element in train and test
train_features = train['filepath'].apply(extract_features)

# train_features is now an array of arrays
test_features = test['filepath'].apply(extract_features)

train_features.head()

8     [-494.1591491699219, 77.5472412109375, 33.6363...
12    [-613.1090087890625, 92.3533706665039, 2.85021...
2     [-754.1896362304688, 64.00310516357422, 7.3716...
4     [-496.4476318359375, 129.58126831054688, 61.19...
6     [-399.6903076171875, 147.39346313476562, 54.82...
Name: filepath, dtype: object

In [70]:
# split into X and Y where X is the features and Y is the label (name of speaker)
# remember that each array is still in the same order as above 
# so each element in X_train corresponds to an element in Y_train at the same index
X_train = train_features.tolist()
X_test = test_features.tolist()
Y_train = train['speaker'].tolist()
Y_test = test['speaker'].tolist()

In [71]:
# now X_train is a 2d array, and each array is the long array of mfccs
print("Speaker: {}".format(Y_train[0]))
print("Features: {}: ".format(X_train[0]))

Speaker: eli
Features: [-494.1591491699219, 77.5472412109375, 33.636329650878906, 15.328489303588867, -0.5756244659423828, 7.3330607414245605, 8.233574867248535, 1.0448485612869263, -1.5833865404129028, -0.4561096429824829, -1.52070152759552, 6.954306125640869]: 


In [72]:

# hot encode y 
lb = LabelEncoder()

Y_train_encoded = to_categorical(lb.fit_transform(Y_train))
Y_test_encoded = to_categorical(lb.fit_transform(Y_test))

In [73]:
ss = StandardScaler()

X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.fit_transform(X_test)


In [74]:
model = Sequential()

model.add(Dense(MFCCS, input_shape=(MFCCS,), activation = 'relu'))
model.add(Dropout(0.2))

# model.add(Dense(256, activation = 'relu'))
# model.add(Dropout(0.01))  

model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.3))    

model.add(Dense(len(Y_train_encoded[0]), activation = 'softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train_scaled, Y_train_encoded, epochs=20, validation_data=(X_test_scaled, Y_test_encoded))

Epoch 1/20
1/1 [==============================] - 0s 125ms/step - loss: 1.4112 - accuracy: 0.2727 - val_loss: 1.2356 - val_accuracy: 0.6000
Epoch 2/20
1/1 [==============================] - 0s 26ms/step - loss: 1.2288 - accuracy: 0.6364 - val_loss: 1.1586 - val_accuracy: 0.4000
Epoch 3/20
1/1 [==============================] - 0s 16ms/step - loss: 1.1171 - accuracy: 0.8182 - val_loss: 1.0911 - val_accuracy: 0.4000
Epoch 4/20
1/1 [==============================] - 0s 17ms/step - loss: 1.0330 - accuracy: 0.7273 - val_loss: 1.0436 - val_accuracy: 0.6000
Epoch 5/20
1/1 [==============================] - 0s 17ms/step - loss: 0.9582 - accuracy: 0.8182 - val_loss: 1.0053 - val_accuracy: 0.4000
Epoch 6/20
1/1 [==============================] - 0s 16ms/step - loss: 0.9069 - accuracy: 0.7273 - val_loss: 0.9677 - val_accuracy: 0.8000
Epoch 7/20
1/1 [==============================] - 0s 16ms/step - loss: 0.8516 - accuracy: 0.9091 - val_loss: 0.9376 - val_accuracy: 0.8000
Epoch 8/20
1/1 [==========

In [75]:
def predict_speaker(file_path):
      
    labels = Y_train_encoded.tolist() # list of the encoded labels from training 
    
    features = ss.transform([extract_features(file_path)])
    to_predict = np.array(features) # list of data for the model to predict, just one item for now
    
    predictions = model.predict(to_predict) # returns a list of predictions 
    pred = predictions[0].tolist() # take the first element which is the prediciton for the first element in to_predict, remember this is still one hot encoded so it is a big array of 0s and 1s
    
    m = max(pred)
    p = [1 if i==m else 0 for i in pred] # convert highest propability to 1 and all else to 0
    
    try:
        prediction_ind = labels.index(p) # index of predicted label (encoded)
        prediction = Y_train[prediction_ind]
    except:
        prediction = "could not identify speaker"
        
    return prediction
    


In [76]:
predict_speaker("test/alex.wav")

'alex'

In [77]:
predict_speaker("test/Harley.wav")

'harley'

In [78]:
predict_speaker("test/eli.wav")

'harley'

In [79]:
predict_speaker("test/alastair.wav")

'alastair'

In [81]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 2
WAVE_OUTPUT_FILENAME = "test/audio_input.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
print("say 'Alexa' in: 3")
for i in range(2,0,-1):
    time.sleep(1)
    print (i)
time.sleep(1)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()
print()
print(predict_speaker(WAVE_OUTPUT_FILENAME))


say 'Alexa' in: 3
2
1
* recording
* done recording

harley
